# Tree Creater

#### Imports

In [ ]:
%pylab inline
%matplotlib inline

import numpy as np
import cv2
import time
import TreeData as td
import pickle
import ImageProcessing as ipr
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display
from sklearn import tree as decisionTree
import BinaryTree as bt
from scipy.ndimage.measurements import label
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydot
import graphviz as gz

#### Load treated data

In [ ]:
pickleFile  = open('my data/' + 'treatedData' + '.pkl', 'rb')
labelDict = pickle.load(pickleFile)
pickleFile.close()

print('Correctly opened ' + str(len(labelDict)) + ' treated training data !')

#### Confusion matrices

In [ ]:
confusionAll = np.zeros((2, 2), dtype='int')
emptyPrediction = np.zeros((len(labelDict[0]['actual']),\
                            len(labelDict[0]['actual'][0])), dtype='uint8')

for i in range(len(labelDict)):
    confusion, stats = td.compute_confusion_matrix(labelDict[i]['actual'],\
                                                   emptyPrediction)
    confusionAll += confusion
    labelDict[i]['matrix'] = confusion
    labelDict[i]['stats'] = stats
    print('Image n°' + str(i+1) + ' (' + str(labelDict[i]['fingerPoints'])\
          + ' finger points):\n' + str(labelDict[i]['matrix'])\
          + '\n' + str(labelDict[i]['stats']) + '\n')
print('All Images: \n' + str(confusionAll) + '\n'\
      + str(td.compute_stats(confusionAll)))

#### Create test and training sets

In [ ]:
trainingExampleIndices = [0, 1, 2, 8]
testExampleIndices = [3, 4, 5, 6, 7]
trainingDict = [labelDict[index]for index in trainingExampleIndices]
testDict = [labelDict[index]for index in testExampleIndices]

print('Successfuly listed ' + str(len(trainingDict)) + ' training images !\nAnd ' + \
      str(len(testDict)) + ' test images !')

In [ ]:
trainX, trainY = bt.extractFromDictionnary(trainingDict)
testX, testY = bt.extractFromDictionnary(testDict)

print('The Test and Training datas were correctly exctracted')

#### Create binary tree

In [ ]:
tree = bt.Tree('4 pix rule')
tree.fit(trainX, trainY, 4)

print('Successfuly created tree \'' + tree.name + '\' !')

#### Visualize optimal rules

In [ ]:
visualRules = tree.show()
if(visualRules != False):
    length = len(visualRules)
    fig = figure(0, (5*length,5*length))
    if (isinstance(tree.optimalRules[0], str) == False):
        for index in range(length):
            subplot(1, length, index+1)
            title(visualRules[index][1])
            imshow(visualRules[index][0], cmap = 'seismic')

In [ ]:
#yHat = tree.predictFromOutsiderRules(testX, tree.endNodes[4].rule[0])
#
#cm, stats = td.compute_confusion_matrix(testY, yHat)
#print(str(len(testY)) + ' pixels were evaluated !')
#print(cm)
#print(stats)

#### Confusion matrix

In [ ]:
yHat = tree.predictFromOptimalRules(testX)

cm, stats = td.compute_confusion_matrix(testY, yHat)
print(str(len(testY)) + ' pixels were evaluated !')
print(cm)
print(stats)

#### Save text trees to visualization

In [ ]:
#for depth in range(1, 6):
#   clf = decisionTree.DecisionTreeClassifier(criterion='entropy', max_depth = depth)
#   clf = clf.fit(trainX, trainY)
#    yHatTraining = clf.predict(trainX)
#    yHatTest = clf.predict(testX)
#    ------------------------------#
#   with open("my data/finger_classifier_" + str(depth) + ".txt", "w") as f:
#       f = decisionTree.export_graphviz(clf, out_file=f)
#------------------------------#
#cm, stats = td.compute_confusion_matrix(testY, yHatTest)
#print(str(len(testY)) + ' pixels were evaluated !')
#print(cm)
#print(stats)

#### Visualize the matches

In [ ]:
fig = figure(0, (30, 15))
column = 5

for i in range(9):
    labelDict[i]['matches'] = ipr.show_matches(tree.optimalRules, labelDict[i])
    subplot(2, column, i+1)
    title('Image n° ' + str(i+1), size='xx-large')
    imshow(labelDict[i]['matches'])

In [ ]:
fig = figure(0, (25, 15))
count = 0
for image in testDict:
    image['predicted'] = yHat[2109*count:2109*(count+1)].reshape(57, 37)
    image['numFingers'] = ipr.countFingers(image['predicted'], 25)
    subplot(3, len(testDict), count+1)
    imshow(image['actual'])
    subplot(3, len(testDict), count+1+len(testDict))
    title(str(image['numFingers']) + ' fingers', size='xx-large')
    imshow(image['predicted'])
    subplot(3, len(testDict), count+1+(2*len(testDict)))
    imshow(image['matches'])
    count += 1

#### Dump optimal rules

In [ ]:
#name = input('File name : ')
#
#dumpingRules = []
#for rule in tree.optimalRules:
#    dumpingRules.append(rule[1:])
#
#pickleFile = open('my data/'+ name + '.pkl', 'wb')
#pickle.dump(dumpingRules, pickleFile, pickle.HIGHEST_PROTOCOL)
#pickleFile.close()